In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


In [ ]:

# Dataset path
dataset_path = '/content/drive/MyDrive/Fraud1/data/train_data'

# Image dimensions
img_height = 224
img_width = 224
batch_size = 32


In [ ]:
def preprocess_grayscale_to_rgb(image):
    """Convert a grayscale image to RGB by replicating the channel."""
    if len(image.shape) == 2 or image.shape[-1] == 1:  # Grayscale image
        image = np.stack((image.squeeze(),) * 3, axis=-1)  # Convert to RGB
    return image


In [ ]:
# Updated Data Generators
datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values
    validation_split=0.2  # Reserve 20% for validation
)

train_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',  # Force output to have 3 channels
    subset='training'
)

val_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',  # Force output to have 3 channels
    subset='validation'
)


Found 24 images belonging to 5 classes.
Found 5 images belonging to 5 classes.


In [ ]:

# Load the pre-trained VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the base model layers
base_model.trainable = False


In [ ]:

# Build the new model
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_gen.num_classes, activation='softmax')  # Output layer matches class count
])


In [ ]:

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:

# Train the model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 21s 21s/step - accuracy: 0.2500 - loss: 2.0787 - val_accuracy: 0.4000 - val_loss: 2.0770
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 19s 19s/step - accuracy: 0.5000 - loss: 2.2291 - val_accuracy: 0.2000 - val_loss: 1.7018
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 23s 23s/step - accuracy: 0.4583 - loss: 2.6393 - val_accuracy: 1.0000 - val_loss: 0.1563
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 23s 23s/step - accuracy: 0.7500 - loss: 0.4417 - val_accuracy: 1.0000 - val_loss: 0.0937
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 37s 37s/step - accuracy: 0.8750 - loss: 0.2488 - val_accuracy: 1.0000 - val_loss: 0.0956
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - accuracy: 0.7917 - loss: 0.6193 - val_accuracy: 1.0000 - val_loss: 0.1476
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 20s 20s/step - accuracy: 0.8750 - loss: 0.4207 - val_accuracy: 1.0000 - val_loss: 0.1968
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - accuracy: 0.7917 - loss: 0.3188 - val_accuracy: 1.0000 - val_loss: 0.1715
Epoch 9/10


In [ ]:

# Save the model for later use
model.save('signature_vgg16_model.h5')


In [ ]:

# Class labels
class_names = list(train_gen.class_indices.keys())
print(f"Class Names: {class_names}")


Class Names: ['anfal', 'dhia', 'lina', 'mehdi', 'raouf']


In [ ]:
def predict_signature(image_path, model, class_names, threshold=0.7):
    """
    Predict the class of a signature image.
    Handles grayscale images and replicates channels to make them RGB.
    """
    from tensorflow.keras.preprocessing.image import load_img, img_to_array

    # Load the grayscale image
    image = load_img(image_path, target_size=(224, 224), color_mode='grayscale')
    image = img_to_array(image)  # Shape: (224, 224, 1)

    # Use the provided function to preprocess the grayscale image to RGB
    image = preprocess_grayscale_to_rgb(image)  # Shape: (224, 224, 3)
    image = image / 255.0  # Normalize pixel values
    image = np.expand_dims(image, axis=0)  # Add batch dimension

    # Perform prediction
    predictions = model.predict(image)
    confidence = np.max(predictions)
    predicted_class = class_names[np.argmax(predictions)]

    # Handle "Unknown or Forgery" case
    if confidence < threshold:
        return {"Prediction": "Unknown or Forgery", "Confidence": confidence}
    return {"Prediction": predicted_class, "Confidence": confidence}



In [ ]:

# Example usage
image_path = '/content/drive/MyDrive/Fraud1/data/test_data/forg15.png'
result = predict_signature(image_path, model, class_names)
print(f"Prediction: {result['Prediction']}, Confidence: {result['Confidence']:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 552ms/step
Prediction: Unknown or Forgery, Confidence: 0.63


In [ ]:
## Working code for all classes
def predict_all_signatures(directory, model, class_names, threshold=0.7):
    """
    Predict the class of all signature images in a directory.
    """
    results = []

    for filename in os.listdir(directory):
        image_path = os.path.join(directory, filename)

        # Check if it's an image file
        if filename.lower().endswith(('png', 'jpg', 'jpeg')):
            result = predict_signature(image_path, model, class_names, threshold)
            results.append({
                "Image": filename,
                "Prediction": result["Prediction"],
                "Confidence": result["Confidence"]
            })

    return results

In [ ]:
from keras.models import load_model
model = load_model('/content/79modelvgg16.h5')

In [ ]:
## Limt to one target class
import os
def predict_single_class_signatures(directory, model, target_class, class_names, threshold=0.7):
    """
    Predict whether all signature images in a directory belong to a single specified class.

    Args:
        directory (str): Path to the directory containing images.
        model (keras.Model): The trained model.
        target_class (str): The class to compare against.
        class_names (list): List of all class names.
        threshold (float): Confidence threshold to decide if the signature matches the target class.

    Returns:
        list: Results for each image in the directory.
    """
    from tensorflow.keras.preprocessing.image import load_img, img_to_array
    results = []

    # Ensure target class is in class_names
    if target_class not in class_names:
        raise ValueError(f"Target class '{target_class}' not found in class names: {class_names}")

    # Get the index of the target class
    target_class_index = class_names.index(target_class)

    for filename in os.listdir(directory):
        image_path = os.path.join(directory, filename)

        # Check if it's an image file
        if filename.lower().endswith(('png', 'jpg', 'jpeg')):
            # Preprocess and predict
            image = load_img(image_path, target_size=(224, 224), color_mode='grayscale')
            image = img_to_array(image)  # Shape: (224, 224, 1)

            # Use the provided function to preprocess the grayscale image to RGB
            image = preprocess_grayscale_to_rgb(image)  # Shape: (224, 224, 3)
            image = image / 255.0  # Normalize pixel values
            image = np.expand_dims(image, axis=0)  # Define or use an existing preprocessing function
            predictions = model.predict(image)[0]  # Predict probabilities for all classes

            confidence = predictions[target_class_index]
            is_match = confidence >= threshold

            # Record results
            results.append({
                "Image": filename,
                "Match": is_match,
                "Confidence": confidence,
                "Target Class": target_class
            })

    return results


In [ ]:

# Example usage
test_directory = '/content/drive/MyDrive/Fraud1/data/forgeries'
all_results = predict_all_signatures(test_directory, model, class_names)

# Print results
for result in all_results:
    print(f"Image: {result['Image']}, Prediction: {result['Prediction']}, Confidence: {result['Confidence']:.2f}")


NameError: name 'predict_all_signatures' is not defined

In [ ]:
directory = "/content/"
target_class = "anfal"  # Compare against this class only
class_names = ['anfal', 'dhia', 'lina', 'mehdi', 'raouf']  # Class names in your model

results = predict_single_class_signatures(directory, model, target_class, class_names, threshold=0.7)

# Print results
for result in results:
    print(f"Image: {result['Image']}, Match: {result['Match']}, Confidence: {result['Confidence']:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 839ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 905ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 565ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 477ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 482ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 500ms/step
Image: LD1.jpg, Match: False, Confidence: 0.29
Image: LD4.jpg, Match: False, Confidence: 0.07
Image: IDK.jpg, Match: False, Confidence: 0.30
Image: LD2.jpg, Match: False, Confidence: 0.13
Image: anfal.jpg, Match: True, Confidence: 0.99
Image: LD3.jpg, Match: False, Confidence: 0.13
Image: ikd_at_this_poing.jpg, Match: True, Confidence: 0.95
